In [6]:
import os

os.environ['WANDB_NOTEBOOK_NAME'] = '/Users/aditya/Programming/stock-market-predictions/stock_market_predictions/forecasting/LSTM-5-Year/MSFT-5-Year-LSTM.ipynb'

In [7]:
def train():
    import pandas as pd
    from os.path import abspath

    import warnings
    warnings.filterwarnings("ignore")
    df = pd.read_csv(abspath('../../../data/MSFT/MSFT-5-Year.csv'), index_col=0, parse_dates=['Date'])

    df.columns.name = df.index.name
    df.index.name = None

    df = df.filter(['Date', 'Open'])
    # Split the df into a training set and a test set
    from sklearn.model_selection import train_test_split

    # Split the data into training and testing sets
    train_set, test_set = train_test_split(df, test_size=0.05, shuffle=False)

    n_features = 1
    from sklearn.preprocessing import MinMaxScaler

    transformer = MinMaxScaler()
    train_scaled = transformer.fit_transform(train_set)
    test_scaled = transformer.fit_transform(test_set)
    
    import tensorflow as tf
    from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # type: ignore

    n_input = 60

    train_generator=TimeseriesGenerator(train_scaled,
                                        train_scaled,
                                        length=n_input,
                                        batch_size=32)
    
    from keras.models import Sequential
    from keras.layers import Dense, LSTM

    LSTM_UNITS = 20

    model = Sequential()
    model.add(LSTM(units=LSTM_UNITS, input_shape=(n_input, n_features)))
    model.add(Dense(units=1))
    
    # Create an EarlyStopping callback
    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    
    import wandb
    from wandb.keras import WandbCallback

    wandb.init(project="stock-predictions", entity="alpheron", magic=True)

    wandb.config = {
    "learning_rate": 0.001,
    "epochs": 100,
    "batch_size": 32,
    "lstm_units": LSTM_UNITS,
    "optimizer": "adam",
    }

    from tensorflow.keras.metrics import RootMeanSquaredError
    from stock_market_predictions.forecasting.adam import Adam

    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[RootMeanSquaredError()])
    model.fit(train_generator, epochs=100, callbacks=[early_stopping, WandbCallback()])
    model.save(abspath('../../../logs/MSFT-5-Year/'))

    inputs = df[len(df)-len(test_set)-60 : ].values
    inputs = inputs.reshape(-1,1)
    inputs = transformer.transform(inputs)
    import numpy as np

    x_test = []
    for i in range(60,len(inputs)):
        x_test.append(inputs[i-60:i, 0])
        
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    print(x_test.shape)
    predicted = model.predict(x_test)
    predicted = transformer.inverse_transform(predicted)

    # Convert the predicted values to a dataframe
    predicted_df = pd.DataFrame(predicted)

    # Rename the column to "Open"
    predicted_df.columns = ['Open']

    # Add a new column called "Date" that has the same index as the other dataframe
    predicted_df['Date'] = df[len(df)-len(test_set)-0 : ].index

    # Set the 'Date' column as the index
    predicted_df = predicted_df.set_index('Date')

    import math
    from sklearn.metrics import mean_squared_error

    def return_rmse(test, predicted):
        rmse = math.sqrt(mean_squared_error(test, predicted))
        print("the root mean squared error is: {}".format(rmse))
        return rmse

    RMSE = return_rmse(test_set, predicted)
    wandb.log({"root_mean_square_error" : RMSE})
    artifact = wandb.Artifact('Time-Series', type='model')
    artifact.add_dir(abspath('../../../logs/MSFT-5-Year/'))
    wandb.run.log_artifact(artifact)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/Caskroom/miniconda/base/envs/StockMarket/lib/python3.8/site-packages/keras/backend.py", line 4739, in <genexpr>
    ta.write(time, out) for ta, out in zip(output_ta_t, flat_output))  File "/usr/local/Caskroom/miniconda/base/envs/StockMarket/lib/python3.8/site-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/Caskroom/miniconda/base/envs/StockMarket/lib/python3.8/site-packages/keras/backend.py", line 4739, in <genexpr>
    ta.write(time, out) for ta, out in zip(output_ta_t, flat_output))  File "/usr/local/Caskroom/miniconda/base/envs/StockMarket/lib/python3.8/site-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


In [8]:
sweep_config = {
    'method': 'bayes',
    'metric': 
        {
            'name': 'root_mean_square_error',
            'goal': 'minimize',
        },
    'parameters': {
        'LSTM_UNITS': {
            'values': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 120, 125, 130, 135, 140, 145, 150]
        },
        'epochs': {
            'value': 100
        }
    },
    'early_terminate': {
        'type': 'hyperband',
        's': 2,
        'eta': 3,
        'max_iter': 50
    }
}

In [9]:
import wandb
# sweep_id = wandb.sweep(sweep_config, project="stock-predictions", entity="alpheron")

In [10]:
wandb.agent('21t67yna', function=train, project="stock-predictions", entity="alpheron")

wandb: Agent Starting Run: jw2yg2d6 with config:
wandb: 	LSTM_UNITS: 85
wandb: 	epochs: 100
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/100
36/36 [==============================] - 8s 231ms/step - loss: 0.0837 - root_mean_squared_error: 0.2894
Epoch 2/100
36/36 [==============================] - 9s 246ms/step - loss: 0.0104 - root_mean_squared_error: 0.1020
Epoch 3/100
36/36 [==============================] - 11s 303ms/step - loss: 0.0029 - root_mean_squared_error: 0.0536
Epoch 4/100
36/36 [==============================] - 11s 300ms/step - loss: 8.4761e-04 - root_mean_squared_error: 0.0291
Epoch 5/100
36/36 [==============================] - 8s 231ms/step - loss: 8.0264e-04 - root_mean_squared_error: 0.0283
Epoch 6/100
36/36 [==============================] - 8s 224ms/step - loss: 7.6498e-04 - root_mean_squared_error: 0.0277
Epoch 7/100
36/36 [==============================] - 6s 172ms/step - loss: 8.8985e-04 - root_mean_squared_error: 0.0298
Epoch 8/100
36/36 [==============================] - 6s 181ms/step - loss: 6.6991e-04 - root_mean_squared_error: 0.0259
Epoch 9/100
36/36 [==============================]

INFO:tensorflow:Assets written to: /Users/aditya/Programming/stock-market-predictions/logs/MSFT-5-Year/assets


INFO:tensorflow:Assets written to: /Users/aditya/Programming/stock-market-predictions/logs/MSFT-5-Year/assets


(63, 60, 1)


...

INFO:tensorflow:Assets written to: /Users/aditya/Programming/stock-market-predictions/logs/MSFT-5-Year/assets
wandb: Adding directory to artifact (/Users/aditya/Programming/stock-market-predictions/logs/MSFT-5-Year)... Done. 0.0s


(63, 60, 1)
the root mean squared error is: 9.601596379533017


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_square_error,▁
root_mean_squared_error,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,30
loss,0.00046
root_mean_square_error,9.6016
root_mean_squared_error,0.02148


wandb: Agent Starting Run: bqpa3hap with config:
wandb: 	LSTM_UNITS: 45
wandb: 	epochs: 100
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/100
36/36 [==============================] - 7s 200ms/step - loss: 0.0301 - root_mean_squared_error: 0.1734
Epoch 2/100
36/36 [==============================] - 5s 150ms/step - loss: 0.0046 - root_mean_squared_error: 0.0677
Epoch 3/100
36/36 [==============================] - 5s 140ms/step - loss: 0.0011 - root_mean_squared_error: 0.0333
Epoch 4/100
36/36 [==============================] - 5s 148ms/step - loss: 4.7066e-04 - root_mean_squared_error: 0.0217
Epoch 5/100
36/36 [==============================] - 5s 142ms/step - loss: 4.0809e-04 - root_mean_squared_error: 0.0202
Epoch 6/100
36/36 [==============================] - 5s 133ms/step - loss: 4.2238e-04 - root_mean_squared_error: 0.0206
Epoch 7/100
36/36 [==============================] - 6s 166ms/step - loss: 5.0786e-04 - root_mean_squared_error: 0.0225
Epoch 8/100
36/36 [==============================] - 5s 138ms/step - loss: 4.0265e-04 - root_mean_squared_error: 0.0201
Epoch 9/100
36/36 [==============================] -